In [9]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split
from statistics import mean
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from gensim.matutils import softcossim
import math
import nltk
lemma = nltk.wordnet.WordNetLemmatizer()
#python -m pip install -U pycld2
import pycld2 as cld2
import spacy
sp = spacy.load("en")
sp_stopwords=sp.Defaults.stop_words
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.downloader as api
from gensim import corpora,models
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
w2v_model = api.load("glove-wiki-gigaword-50")

Pomoćne funkcije

In [12]:
def union (list1, list2):
    final_list = list(set(list1) | set(list2)) 
    return final_list 

def isNaN(string):
    return string != string
def acronyms_to_words (txt):
    #acronyms = pd.read_excel("/Users/Lukre/Desktop/D/kratice.xlsx")
    #acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})
    for i in range (len(acronyms)):
        txt=txt.replace(' ' + acronyms.acronym[i] + ' ',' ' + acronyms.replacement[i] + ' ')
    return txt

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Priprema teksta:  

    -obrada i modifikacija u listu tokena
    -obrada s povratnom vrijednosti tipa 'str' 

In [13]:
def tokenizing_text(txt):
    tok=[]
    pom=txt.replace('. ',' ')
    pom=txt.replace('.',' ')
    pom=txt.replace(',',' ')
    pom=txt.replace(';',' ')
    pom=txt.replace(':',' ')
    pom=txt.replace('?',' ')
    pom=txt.replace('!',' ')
    pom=pom.lower()
    pom=acronyms_to_words(pom)
    sp_pom=sp(pom)
    sp_pom=[x for x in sp_pom if not x.is_stop and x.is_alpha]
    for w in sp_pom:
        tok.append(w.lemma_)
    return tok
def prep_text(txt):
    txt=txt.replace('. ',' ')
    txt=txt.replace('.',' ')
    txt=txt.replace(',',' ')
    txt=txt.replace(';',' ')
    txt=txt.replace(':',' ')
    txt=txt.replace('?',' ')
    txt=txt.replace('!',' ')
    txt=txt.lower()
    txt=acronyms_to_words(txt)
    sp_pom=sp(txt)
    sp_pom=[x for x in sp_pom if not x.is_stop and x.is_alpha]
    pom=""
    for w in sp_pom:
        pom=pom + " " + w.lemma_
    txt=pom
    return txt

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Obrada svih dokumenata sa prep_text

In [7]:
def prep_doc(document,risk,token):
    
    if (risk=='risk'):
        doc = pd.DataFrame(columns=['xx', 'report', 'label','factor', 'accident'])
    else:
        doc = pd.DataFrame(columns=['xx', 'report'])
    
    for i in range(len(document)):
       
        if isNaN(document.report[i]):
            continue
        isReliable, textBytesFound, details=cld2.detect(document.report[i])
        if not isReliable:
            continue
        if details[0][0]!='ENGLISH':
            continue
        if (token=='tok'):
            txt=tokenizing_text(document.report[i])
        else:
            txt=prep_text(document.report[i])
            
        if (risk=='risk'):
            if (document.factor[i]==1):
                pom='no accident outcome'
            elif (document.factor[i] in (2,4,20,100)):
                pom='minor injuries or damage'
            else:
                pom='major or catastrophic accident'
            
            doc=doc.append({'xx': document.xx[i], 'report': txt, 
                    'label': document.label[i], 'factor': document.factor[i],
                        'accident': pom},ignore_index=True)
        
        else:
            doc=doc.append({'xx': document.xx[i], 'report': txt}, ignore_index=True) 
    return doc

Obrada svih dokumenata i modifikacija u tokene    
  
  &emsp;fja vraća:  
  &emsp;&emsp;-skup svih tokena sa brojem dokumenata u kojima se pojedini token pojavljuje  
  &emsp;&emsp;-DataFrame u kojem svaki redak označava jednu kombinaciju dokument-token i tfidf vrijednost te kombinacije

In [14]:
def tokenizing_Data(doc):
    j=0
    br=-1
    tokens_list=[]
    tokens=pd.DataFrame()
    TD=pd.DataFrame(columns=['doc','tok','tf_idf'])
    n_data=len(doc)
    
    for i in range (n_data):
        
        if isNaN(doc.report[i]):
            continue
        
        #provjerava je li izvjestaj na ENG jeziku
        isReliable, textBytesFound, details=cld2.detect(doc.report[i])
        
        if not isReliable:
            continue
        if details[0][0]!='ENGLISH':
            continue
            
        br=br+1;
        tokenized_text=tokenizing_text(doc.report[i])
        n_tok=len(tokenized_text)
        
        if br==0:
            tokens_list=union(tokens_list,tokenized_text)
            for w in tokens_list:
                tokens[w]=0
            tokens.loc[0] = 0
            for w in tokens_list:
                nmb=tokenized_text.count(w)
                tokens.loc[[0],[w]] = tokens.loc[[0],[w]] + 1
                TD.loc[j]=0
                TD.doc[j] = doc.xx[i]
                TD.tok[j] = w
                TD.tf_idf[j] = nmb/n_tok
                j=j+1
        else:
            pom_tok=union([],tokenized_text)
            for w in pom_tok:
                nmb=tokenized_text.count(w)
                if w not in tokens_list:
                    tokens[w] = 0
                tokens.loc[[0],[w]] = tokens.loc[[0],[w]] + 1
                    
                TD.loc[j]=0
                TD.doc[j] = doc.xx[i]
                TD.tok[j] = w
                TD.tf_idf[j] = nmb/n_tok
                j=j+1
            tokens_list=union(tokens_list,pom_tok)
        
        if (i%100==0):
            print(i)
    
    N=br+1 
    print('j=',j)
    for k in range (j):
        pom=TD.tf_idf[k]
        pom2 = TD.tok[k]
        df=pd.to_numeric(tokens[pom2])
        l =1+ N/df
        l = math.log(l)
        TD.tf_idf[k] = pom*l
        if (k%1000==0):
            print(k)
        
    return(tokens,TD)

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fja prima:  
&emsp;-DataFrame sa document-token kombinacijama  
Fja vraća:  
&emsp;-rijetku matricu sa tfidf vrijednostima svih document-token kombinacija

In [6]:
def pdoc2sparsedf(pdoc):
    n=len(set(pdoc.tok))
    m=len(set(pdoc.doc))
    zero = np.zeros(shape=(m,n))
    df=pd.DataFrame(zero,columns=[list(set(pdoc.tok))],index=[list(set(pdoc.doc))])
    for i in range (len(pdoc)):
        if (i%1000==0):
            print(i)
        a=[pdoc.doc[i]]
        b=[pdoc.tok[i]]
        #df.set_value(a, b, pdoc.tf_idf[i])
        df.loc[a,b]=pdoc.tf_idf[i]
    return df

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fja reducira sve tokene koji se pojavljuju u manje od 8 dokumenata  
&emsp;Fja vraća:  
&emsp;&emsp;-reducirani DataFrame sa svim preostalim document-token kombinacijama i njihovim tfidf vrijednostima  
&emsp;&emsp;-DataFrame preostalih tokena sa brojem dokumenata u kojima se pojavljuju  
&emsp;&emsp;-listu odbačenih tokena

In [7]:
def pdoc_reduced(pdoc,tokens):
    new_pdoc = pd.DataFrame(columns=['doc','tok','tf_idf'])
    new_tokens = pd.DataFrame()
    ejected_tokens = []
    n=len(pdoc)
    
    for col in tokens.columns:
        if (tokens.loc[0][col] < 8):
            ejected_tokens.append(col)
            continue
        else:
            new_tokens[col]=0
            
    new_tokens.loc[0]=0        
    for col in new_tokens.columns:
        new_tokens.loc[0][col]=tokens.loc[0][col]
        
    br=0
    for i in range(n):
        if (i%1000==0):
            print(i)
            
        if (pdoc.tok[i] in new_tokens.columns):
            new_pdoc.loc[br]=0
            new_pdoc.doc[br]=pdoc.doc[i]
            new_pdoc.tok[br]=pdoc.tok[i]
            new_pdoc.tf_idf[br]=pdoc.tf_idf[i]
            tf=len(pdoc[pdoc.doc==pdoc.doc[i]])
            br+=1
    print ("ima ih ", br)
   
    for i in range(len(new_pdoc)):
        if (i%1000==0):
            print(i)
        tf_idf=new_pdoc.tf_idf[i]
        tf_idf=(tf_idf * len(new_pdoc[new_pdoc.doc==new_pdoc.doc[i]])) / len(pdoc[pdoc.doc==pdoc.doc[i]])
        new_pdoc.tf_idf[i]=tf_idf
        
    return new_pdoc, new_tokens , ejected_tokens

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Učitavanje dokumenata

In [10]:
document = pd.read_csv("/Users/Lukre/Desktop/D_dipl/qa06_all.csv")
document = document.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report'})
document_risk = pd.read_csv("/Users/Lukre/Desktop/D_dipl/qa06_only_having_risk_valuesl.csv")
document_risk = document_risk.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report','ty26colo':'label','ty26fakt':'factor'})

acronyms = pd.read_excel("/Users/Lukre/Desktop/D_dipl/kratice.xlsx")
acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Priprema dokumenata

In [ ]:
doc_risk_tok=prep_doc(document_risk,'risk','tok')
doc_risk_str=prep_doc(document_risk,'risk','str')
doc_no_risk_tok=prep_doc(document,'no risk','tok')
doc_no_risk_str=prep_doc(document,'no risk','str')

pdoc_risk=tokenizing_Data(document_risk)
tokens_risk=pdoc_risk[0]
pdoc_risk=pdoc_risk[1]

pdoc_no_risk=tokenizing_Data(document)
tokens_no_risk=pdoc_no_risk[0]
pdoc_no_risk=pdoc_no_risk[1]

sparse_risk=pdoc2sparsedf(pdoc_risk)
sparse_no_risk=pdoc2sparsedf(pdoc_no_risk)

pdoc_risk_red=pdoc_reduced(pdoc_risk,tokens_risk)
tokens_risk_red=pdoc_risk_red[1]
ejected_tokens_risk=pdoc_risk_red[2]
pdoc_risk_red=pdoc_risk_red[0]

pdoc_no_risk_red=pdoc_reduced(pdoc_no_risk,tokens_no_risk)
tokens_no_risk_red=pdoc_no_risk_red[1]
ejected_tokens_no_risk=pdoc_no_risk_red[2]
pdoc_no_risk_red=pdoc_no_risk_red[0]

sparse_risk_red=pdoc2sparsedf(pdoc_risk_red)
sparse_no_risk_red=pdoc2sparsedf(pdoc_no_risk_red)

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Zapisivanje obrada dokumenata u .csv datoteke

In [ ]:
doc_risk_tok.to_csv('/Users/Lukre/Desktop/D_dipl/doc_risk_tok.csv')
doc_risk_str.to_csv('/Users/Lukre/Desktop/D_dipl/doc_risk_str.csv')
doc_no_risk_tok.to_csv('/Users/Lukre/Desktop/D_dipl/doc_no_risk_tok.csv')
doc_no_risk_str.to_csv('/Users/Lukre/Desktop/D_dipl/doc_no_risk_str.csv')
pdoc_risk.to_csv('/Users/Lukre/Desktop/D_dipl/pdoc_risk.csv')
tokens_risk.to_csv('/Users/Lukre/Desktop/D_dipl/tokens_risk.csv')
pdoc_no_risk.to_csv('/Users/Lukre/Desktop/D_dipl/pdoc_no_risk.csv')
tokens_no_risk.to_csv('/Users/Lukre/Desktop/D_dipl/tokens_no_risk.csv')
sparse_risk.to_csv('/Users/Lukre/Desktop/D_dipl/sparse_risk.csv')
sparse_no_risk.to_csv('/Users/Lukre/Desktop/D_dipl/sparse_no_risk.csv')
pdoc_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/pdoc_risk_red.csv')
tokens_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/tokens_risk_red.csv')
pd.DataFrame(ejected_tokens_risk).to_csv('/Users/Lukre/Desktop/D_dipl/ejected_tokens_risk.csv')
pdoc_no_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/pdoc_no_risk_red.csv')
tokens_no_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/tokens_no_risk_red.csv')
pd.DataFrame(ejected_tokens_no_risk).to_csv('/Users/Lukre/Desktop/D_dipl/ejected_tokens_no_risk.csv')
sparse_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/sparse_risk_red.csv')
sparse_no_risk_red.to_csv('/Users/Lukre/Desktop/D_dipl/sparse_no_risk_red.csv')